# Backend

`ebcc` has partial support for a number of tensor backends. The default behaviour simply uses `numpy`, and performs tensor contractions with `numpy.einsum(..., optimize=True)`. When using the `numpy` backend, one can also make use of a custom transpose-transpose-GEMM-transpose `einsum` algorithm, or use TBLIS to perform the contractions.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="cc-pvtz", verbose=0)
mf = scf.RHF(mol).run()

In [2]:
import sys
from logging import StreamHandler
import ebcc

log = ebcc.core.logging.Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

In [3]:
ccsd = ebcc.REBCC(mf, ansatz="CCSD", log=log)
ccsd.kernel()

print("T1 type:", type(ccsd.t1))

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.4000223104    -109.3830288427
   1    -0.3817788205    -109.3647853528     1.824e-02     1.898e-02
   2    -0.3978574865    -109.3808640188     1.608e-02     5.841e-03
   3    -0.3960854698    -109.3790920020     1.772e-03     2.377e-03
   4    -0.3979303867    -109.3809369190     1.845e-03     1.819e-03
   5    -0.3979354486    -109.3809419808     5.062e-06     3.082e-04
   6    -0.3979302342    -109.3809367665

Next, try using TBLIS for the contractions, still with the `numpy` backend.

In [4]:
ebcc.util.einsumfunc.CONTRACTION_METHOD = "tblis"

ccsd = ebcc.REBCC(mf, ansatz="CCSD", log=log)
ccsd.kernel()

print("T1 type:", type(ccsd.t1))


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.4000223104    -109.3830288427
   1    -0.3817788205    -109.3647853528     1.824e-02     1.898e-02
   2    -0.3978574865    -109.3808640188     1.608e-02     5.841e-03
   3    -0.3960854698    -109.3790920020     1.772e-03     2.377e-03
   4    -0.3979303867    -109.3809369190     1.845e-03     1.819e-03
   5    -0.3979354486    -109.3809419808     5.062e-06     3.082e-04
   6    -0.3979302342    -109.3809367665     5.214e-06     3.022e-05
   7    -0.3979380401    -109.3809445723     7.806e-06     1.329e-05
   8    -0.3979371358    -109.3809436681     9.043e-07     1.660e-06
   9    -0.3979370943    -109.3809436265     4.153e-08     1.616e-07
  10    -0.3979371132    -109.3809436455     1.893e-08     5.962e-08

We can also use the Cyclops Tensor Framework (CTF) if the package has been installed.

In [5]:
import sys
for key in list(sys.modules):
    if key.startswith("ebcc"):
        del sys.modules[key]

import os
os.environ["EBCC_BACKEND"] = "ctf"

import ebcc

ccsd = ebcc.REBCC(mf, ansatz="CCSD", log=log)
ccsd.kernel()

print("T1 type:", type(ccsd.t1))

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (1o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.4000223104    -109.3830288427
   1    -0.3817788205    -109.3647853528     1.824e-02     1.898e-02
   2    -0.3978574865    -109.3808640188     1.608e-02     5.841e-03
   3    -0.3960854698    -109.3790920020     1.772e-03     2.377e-03
   4    -0.3979303867    -109.3809369190     1.845e-03     1.819e-03
   5    -0.3979354486    -109.3809419808     5.062e-06     3.082e-04
   6    -0.3979302342    -109.3809367665

When using the `jax` backend, if `pyscfad` is also installed then the auto-differentiation functionality can be used to evaluate gradients. Interfacing with `pyscfad` offers a wide range of functionality beyond this example, and users should refer to the examples in the `pyscfad` repository for more advanced functionality. Note that by default the damping class converts objects standard `numpy` arrays as the array construction and linear algebra for this stop presents a bottleneck for some backends, including `jax`, and is often not necessary to be part of the trace. This behaviour can be controlled using the `ebcc.core.damping.USE_BACKEND` flag, which is `False` by default.

In [7]:
import sys
for key in list(sys.modules):
    if key.startswith("pyscf"):
        del sys.modules[key]
    if key.startswith("ebcc"):
        del sys.modules[key]

import os
os.environ["EBCC_BACKEND"] = "jax"

import warnings
warnings.simplefilter("ignore", UserWarning)

import jax
import ebcc
from pyscfad import gto, scf

mol = gto.Mole(atom="N 0 0 0; N 0 0 1.1", basis="cc-pvtz", verbose=0)
mol.build()
mf = scf.RHF(mol).run()

ccsd = ebcc.REBCC(mf, ansatz="CCSD", log=log)
ccsd.kernel()

print("T1 type:", type(ccsd.t1))

def ccsd(mol):
    mf = scf.RHF(mol)
    mf.kernel()
    ccsd = ebcc.REBCC(mf, ansatz="CCSD", log=ebcc.NullLogger())
    ccsd.kernel()
    return ccsd.e_tot

jacobian = jax.jacrev(ccsd)(mol)
print(f"Nuclear gradient:\n", jacobian.coords)
print(f"Basis exponent gradient:\n", jacobian.exp)

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.4000223104    -109.3830288427
   1    -0.3817788205    -109.3647853528     1.824e-02     1.930e-02
   2    -0.3978574865    -109.3808640188     1.608e-02     9.003e-03
   3    -0.3960854698    -109.3790920020     1.772e-03     2.418e-03
   4    -0.3979303867    -109.3809369190     1.845e-03     2.816e-03
   5    -0.3979354486    -109.3809419808     5.062e-06     5.149e-04
   6    -0.3979302342    -109.3809367665

Also available are `tensorflow` and `cupy`, the latter requiring the platform to have a valid CUDA installation and a GPU available.